In [1]:
import pandas as pd 
rock_samples = pd.read_csv('data/rocksamples.csv') 
rock_samples.head()

,ID,Mission,Type,Subtype,Weight(g),Pristine(%)
0,10001,Apollo11,Soil,Unsieved,125.8,88.36
1,10002,Apollo11,Soil,Unsieved,5629.0,93.73
2,10003,Apollo11,Basalt,Ilmenite,213.0,65.56
3,10004,Apollo11,Core,Unsieved,44.8,71.76
4,10005,Apollo11,Core,Unsieved,53.4,40.31


In [2]:
rock_samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2229 entries, 0 to 2228
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           2229 non-null   int64  
 1   Mission      2229 non-null   object 
 2   Type         2229 non-null   object 
 3   Subtype      2226 non-null   object 
 4   Weight(g)    2229 non-null   float64
 5   Pristine(%)  2229 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 104.6+ KB


In [3]:
rock_samples['Weight(g)'] = rock_samples['Weight(g)'].apply(lambda x : x * 0.001)
rock_samples.rename(columns={'Weight(g)':'Weight(kg)'}, inplace=True)
rock_samples.head()

,ID,Mission,Type,Subtype,Weight(kg),Pristine(%)
0,10001,Apollo11,Soil,Unsieved,0.1258,88.36
1,10002,Apollo11,Soil,Unsieved,5.6290,93.73
2,10003,Apollo11,Basalt,Ilmenite,0.2130,65.56
3,10004,Apollo11,Core,Unsieved,0.0448,71.76
4,10005,Apollo11,Core,Unsieved,0.0534,40.31


In [4]:
missions = pd.DataFrame()
missions['Mission'] = rock_samples['Mission'].unique()
missions.head()

,Mission
0,Apollo11
1,Apollo12
2,Apollo14
3,Apollo15
4,Apollo16


In [5]:
missions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Mission  6 non-null      object
dtypes: object(1)
memory usage: 176.0+ bytes


In [7]:
# Sum total sample weight by mission
sample_total_weight = rock_samples.groupby('Mission')['Weight(kg)'].sum()
missions = pd.merge(missions, sample_total_weight, on='Mission')
missions.rename(columns={'Weight(kg)':'Sample Weight(kg)'}, inplace=True)
missions

,Mission,Sample Weight(kg)
0,Apollo11,21.55424
1,Apollo12,34.34238
2,Apollo14,41.83363
3,Apollo15,75.39910
4,Apollo16,92.46262
5,Apollo17,109.44402


In [8]:
# rock_samples.groupby('Mission') - This groups all the rows by the values in the Mission column.
#rock_samples.groupby('Mission')['Weight(kg)'] - This grabs all the values in the Weight(kg) column, but groups by unique values in the Mission column.
#rock_samples.groupby('Mission')['Weight(kg)'].sum() - This sums all the values in the Weight(kg) column for each unique value in the Mission column.

In [9]:
sample_total_weight = rock_samples.groupby('Mission')['Weight(kg)'].sum()
sample_total_weight

Mission
Apollo11     21.55424
Apollo12     34.34238
Apollo14     41.83363
Apollo15     75.39910
Apollo16     92.46262
Apollo17    109.44402
Name: Weight(kg), dtype: float64

In [10]:
# The next line, pd.merge(missions, sample_total_weight, on='Mission'), can be described as:

# Merge the missions dataframe with the sample_total_weight series by using the Mission column as the index to merge on. What the computer will do is basically this: for each value in the Missions column in the missions dataframe, find that same value in the sample_total_weight series, and add the value from the series into the row as a new column in the dataframe.

In [11]:
# Get the difference in weights across missions
missions['Weight Diff'] = missions['Sample Weight(kg)'].diff()
missions

,Mission,Sample Weight(kg),Weight Diff
0,Apollo11,21.55424,NaN
1,Apollo12,34.34238,12.78814
2,Apollo14,41.83363,7.49125
3,Apollo15,75.39910,33.56547
4,Apollo16,92.46262,17.06352
5,Apollo17,109.44402,16.98140


In [12]:
#Looked only at the Weight Diff column in the missions dataframe
#Filled all "na" (or null) values with a certain value
#The value to fill in the na values is 0
#Saved the modified list of values for that column back into the column
missions['Weight Diff'] = missions['Weight Diff'].fillna(value=0)
missions

,Mission,Sample Weight(kg),Weight Diff
0,Apollo11,21.55424,0.00000
1,Apollo12,34.34238,12.78814
2,Apollo14,41.83363,7.49125
3,Apollo15,75.39910,33.56547
4,Apollo16,92.46262,17.06352
5,Apollo17,109.44402,16.98140


In [14]:
# Add in command and lunar module data
# Fill in any NaN values with 0:
missions['Lunar Module (LM)'] = {'Eagle (LM-5)', 'Intrepid (LM-6)', 'Antares (LM-8)', 'Falcon (LM-10)', 'Orion (LM-11)', 'Challenger (LM-12)'}
missions['LM Mass (kg)'] = {15103, 15235, 15264, 16430, 16445, 16456}
missions['LM Mass Diff'] = missions['LM Mass (kg)'].diff()
missions['LM Mass Diff'] = missions['LM Mass Diff'].fillna(value=0)

missions['Command Module (CM)'] = {'Columbia (CSM-107)', 'Yankee Clipper (CM-108)', 'Kitty Hawk (CM-110)', 'Endeavor (CM-112)', 'Casper (CM-113)', 'America (CM-114)'}
missions['CM Mass (kg)'] = {5560, 5609, 5758, 5875, 5840, 5960}
missions['CM Mass Diff'] = missions['CM Mass (kg)'].diff()
missions['CM Mass Diff'] = missions['CM Mass Diff'].fillna(value=0)

missions

,Mission,Sample Weight(kg),Weight Diff,Lunar Module (LM),LM Mass (kg),LM Mass Diff,Command Module (CM),CM Mass (kg),CM Mass Diff
0,Apollo11,21.55424,0.00000,Orion (LM-11),15264,0.0,Endeavor (CM-112),5960,0.0
1,Apollo12,34.34238,12.78814,Antares (LM-8),15235,-29.0,America (CM-114),5609,-351.0
2,Apollo14,41.83363,7.49125,Intrepid (LM-6),16456,1221.0,Columbia (CSM-107),5840,231.0
3,Apollo15,75.39910,33.56547,Eagle (LM-5),16430,-26.0,Yankee Clipper (CM-108),5875,35.0
4,Apollo16,92.46262,17.06352,Falcon (LM-10),16445,15.0,Kitty Hawk (CM-110),5560,-315.0
5,Apollo17,109.44402,16.98140,Challenger (LM-12),15103,-1342.0,Casper (CM-113),5758,198.0


In [15]:
# We can add some totals for each mission across both the lunar and command modules:
missions['Total Weight (kg)'] = missions['LM Mass (kg)'] + missions['CM Mass (kg)']
missions['Total Weight Diff'] = missions['LM Mass Diff'] + missions['CM Mass Diff']
missions

,Mission,Sample Weight(kg),Weight Diff,Lunar Module (LM),LM Mass (kg),LM Mass Diff,Command Module (CM),CM Mass (kg),CM Mass Diff,Total Weight (kg),Total Weight Diff
0,Apollo11,21.55424,0.00000,Orion (LM-11),15264,0.0,Endeavor (CM-112),5960,0.0,21224,0.0
1,Apollo12,34.34238,12.78814,Antares (LM-8),15235,-29.0,America (CM-114),5609,-351.0,20844,-380.0
2,Apollo14,41.83363,7.49125,Intrepid (LM-6),16456,1221.0,Columbia (CSM-107),5840,231.0,22296,1452.0
3,Apollo15,75.39910,33.56547,Eagle (LM-5),16430,-26.0,Yankee Clipper (CM-108),5875,35.0,22305,9.0
4,Apollo16,92.46262,17.06352,Falcon (LM-10),16445,15.0,Kitty Hawk (CM-110),5560,-315.0,22005,-300.0
5,Apollo17,109.44402,16.98140,Challenger (LM-12),15103,-1342.0,Casper (CM-113),5758,198.0,20861,-1144.0


In [16]:
# We know that the Saturn V payload was 43,500 kg, and the weights of the modules varied from mission to mission. So, to determine the ratios that will allow us to make predictions about the Artemis missions, we can use:

# Saturn V payload
# Mission sample weight
# Mission module weight

# Sample-to-weight ratio
saturnVPayload = 43500
missions['Crewed Area : Payload'] = missions['Total Weight (kg)'] / saturnVPayload
missions['Sample : Crewed Area'] = missions['Sample Weight(kg)'] / missions['Total Weight (kg)']
missions['Sample : Payload'] = missions['Sample Weight(kg)'] / saturnVPayload
missions

,Mission,Sample Weight(kg),Weight Diff,Lunar Module (LM),LM Mass (kg),LM Mass Diff,Command Module (CM),CM Mass (kg),CM Mass Diff,Total Weight (kg),Total Weight Diff,Crewed Area : Payload,Sample : Crewed Area,Sample : Payload
0,Apollo11,21.55424,0.00000,Orion (LM-11),15264,0.0,Endeavor (CM-112),5960,0.0,21224,0.0,0.487908,0.001016,0.000495
1,Apollo12,34.34238,12.78814,Antares (LM-8),15235,-29.0,America (CM-114),5609,-351.0,20844,-380.0,0.479172,0.001648,0.000789
2,Apollo14,41.83363,7.49125,Intrepid (LM-6),16456,1221.0,Columbia (CSM-107),5840,231.0,22296,1452.0,0.512552,0.001876,0.000962
3,Apollo15,75.39910,33.56547,Eagle (LM-5),16430,-26.0,Yankee Clipper (CM-108),5875,35.0,22305,9.0,0.512759,0.003380,0.001733
4,Apollo16,92.46262,17.06352,Falcon (LM-10),16445,15.0,Kitty Hawk (CM-110),5560,-315.0,22005,-300.0,0.505862,0.004202,0.002126
5,Apollo17,109.44402,16.98140,Challenger (LM-12),15103,-1342.0,Casper (CM-113),5758,198.0,20861,-1144.0,0.479563,0.005246,0.002516


In [17]:
#Save the ratios
# We can then use the mean() function to take the average of all those ratios across all the missions.

crewedArea_payload_ratio = missions['Crewed Area : Payload'].mean()
sample_crewedArea_ratio = missions['Sample : Crewed Area'].mean()
sample_payload_ratio = missions['Sample : Payload'].mean()
print(crewedArea_payload_ratio)
print(sample_crewedArea_ratio)
print(sample_payload_ratio)

0.49630268199233724
0.0028946732226251396
0.0014369195019157093


In [18]:
# Predict Artemis sample capacity
# average of two predictions
artemis_crewedArea = 26520
artemis_mission = pd.DataFrame({'Mission':['artemis1','artemis1b','artemis2'],
                                 'Total Weight (kg)':[artemis_crewedArea,artemis_crewedArea,artemis_crewedArea],
                                 'Payload (kg)':[26988, 37965, 42955]})
artemis_mission['Sample Weight from Total (kg)'] = artemis_mission['Total Weight (kg)'] * sample_crewedArea_ratio
artemis_mission['Sample Weight from Payload (kg)'] = artemis_mission['Payload (kg)'] * sample_payload_ratio
artemis_mission['Estimated Sample Weight (kg)'] = (artemis_mission['Sample Weight from Payload (kg)'] + artemis_mission['Sample Weight from Total (kg)'])/2
artemis_mission

,Mission,Total Weight (kg),Payload (kg),Sample Weight from Total (kg),Sample Weight from Payload (kg),Estimated Sample Weight (kg)
0,artemis1,26520,26988,76.766734,38.779584,57.773159
1,artemis1b,26520,37965,76.766734,54.552649,65.659691
2,artemis2,26520,42955,76.766734,61.722877,69.244806


In [19]:
# Prioritize Moon rock sample gathering based on data
rock_samples['Remaining(kg)'] = rock_samples['Weight(kg)'] * (rock_samples['Pristine(%)'] * .01)
low_samples = rock_samples.loc[(rock_samples['Weight(kg)'] >= .16) & (rock_samples['Pristine(%)'] <= 50)]
needed_samples = low_samples[low_samples['Type'].isin(['Basalt', 'Breccia'])]
needed_samples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 11 to 2183
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             22 non-null     int64  
 1   Mission        22 non-null     object 
 2   Type           22 non-null     object 
 3   Subtype        22 non-null     object 
 4   Weight(kg)     22 non-null     float64
 5   Pristine(%)    22 non-null     float64
 6   Remaining(kg)  22 non-null     float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.4+ KB


In [23]:
# Develop a recommendation of Moon rock samples to be collected
needed_samples_overview = pd.DataFrame()
needed_samples_overview['Type'] = needed_samples.Type.unique()
needed_sample_weights = needed_samples.groupby('Type')['Weight(kg)'].sum().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, needed_sample_weights, on='Type')
needed_samples_overview.rename(columns={'Weight(kg)':'Total Weight(kg)'}, inplace=True)
needed_sample_ave_weights = needed_samples.groupby('Type')['Weight(kg)'].mean().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, needed_sample_ave_weights, on='Type')
needed_samples_overview.rename(columns={'Weight(kg)':'Ave Weight(kg)'}, inplace=True)
total_rock_count = rock_samples.groupby('Type')['ID'].count().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, total_rock_count, on='Type')
needed_samples_overview.rename(columns={'ID':'Number of Samples'}, inplace=True)
total_rocks = needed_samples_overview['Number of Samples'].sum()
needed_samples_overview['Percentage of Rocks'] = needed_samples_overview['Number of Samples'] / total_rocks
needed_samples_overview

,Type,Total Weight(kg),Ave Weight(kg),Number of Samples,Percentage of Rocks
0,Basalt,17.4234,1.244529,351,0.267939
1,Breccia,10.1185,1.264812,959,0.732061


In [ ]:
# from this we can say that Over 825 lb, or 375 kg total weight of the samples that were brought back by the six Apollo missions that landed on the Moon.